In [970]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from pydub import AudioSegment

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import MaxAbsScaler

##### Função recebe a quantidade de arquivos por classe, o nome da pasta e o nome das classes. Cria o caminho e salva na pasta lista_arquivos. Depois percorre os arquivos, abrindo cada aúdio, extraindo cada informação, aplica um filtro para remover os outliers, deixando apenas as informações menores que -20 e maiores que 20. Sabendo da necessidade de deixar todos os audios com o mesmo tamanho de array, uso a formula int(shape[0]) / 1000, e pego 1000 dados de cada audio, de forma a percorre-los 

In [971]:
def transform_audios(quantidade_arquivos, pasta, info):

    lista_arquivos = []

    lista_audios = []

    lista_classe = []

    for palavra in info:

        for i in range(quantidade_arquivos):
    
            string = f"{pasta}/{0}{i+1}_{palavra}.mp3"
            lista_arquivos.append(string)

            if palavra == "abrir":
                lista_classe.append(0)

            elif palavra == "compilar":
                lista_classe.append(1)
            
            elif palavra == "executar":
                lista_classe.append(2)

            else:
                lista_classe.append(3)

    for arquivo in lista_arquivos:
    
        sound = AudioSegment.from_mp3(arquivo)
        raw_data = sound._data

        audio = []
        
        for si in range(len(raw_data)//2):
            sample = int.from_bytes(raw_data[si * 2 : si * 2 + 2], 'little', signed=True)
            if sample < -30 or sample > 30:
                audio.append(sample)
        
        audio = np.array(audio)
        
        shape = audio.shape
  
        calculo = int(shape[0]) / 5000

        calculo = int(calculo)
        
        audio = audio[::calculo]
        
        audio = audio[:5000]
        
        lista_audios.append(audio)

    df = pd.DataFrame(lista_audios)
    df["classe"] = lista_classe

    return df

##### Aplicando a função nos arquivos de treinamento e criando x e y para treinamento. Aplicada a função MaxAbsScaler, para normalizar os dados.

In [972]:
quantidade_arquivos_por_classe = 8

info = ["abrir", "compilar", "executar", "testar"]

pasta = "audios_treinamento"

arquivos_audio_treinamento = transform_audios(quantidade_arquivos_por_classe, pasta, info)

In [989]:
x = arquivos_audio_treinamento.drop(['classe'], axis= 1)

In [991]:
y = arquivos_audio_treinamento["classe"]

In [992]:
scaler = MaxAbsScaler().fit(x)  

x = scaler.transform(x)

##### Aplicando a função nos arquivos de teste e criando x_test e y_test, também aplicada função para normalização.

In [993]:
quantidade_arquivos_por_classe = 2

info = ["abrir", "compilar", "executar", "testar"]

pasta = "audios_teste"

arquivos_audio_teste = transform_audios(quantidade_arquivos_por_classe ,pasta, info)

In [994]:
x_test = arquivos_audio_teste.drop(['classe'], axis= 1)

In [996]:
y_test = arquivos_audio_teste["classe"] 

In [997]:
scaler = MaxAbsScaler().fit(x_test)

x_test = scaler.transform(x_test)

##### Treinando algoritmo Knn

In [998]:
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(x, y)

KNeighborsClassifier(n_neighbors=3)

##### testando com os arquivos de teste.

In [999]:
y_pred = knn_model.predict(x_test)

print(f"Testando com dados de teste. Score: {knn_model.score(x_test, y_test) * 100} %")

Testando com dados de teste. Score: 37.5 %


##### Treinando o algoritmo, Embaralhando os dados e testando com dados de treinamento.

In [1000]:
knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(x, y)

KNeighborsClassifier(n_neighbors=1)

In [1001]:
arquivos_audio_teste = arquivos_audio_treinamento.sort_values(0) 

In [1002]:
x_test = arquivos_audio_teste.drop(['classe'], axis= 1)
y_test = arquivos_audio_teste["classe"] 

In [1003]:
y_pred = knn_model.predict(x_test)

print(f"Testando com dados de treinamento. Score: {knn_model.score(x_test, y_test) * 100} %")

Testando com dados de treinamento. Score: 100.0 %


In [1004]:
y_pred = pd.DataFrame(y_pred)

y_pred = y_pred.astype(int)

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         8
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00         8

    accuracy                           1.00        32
   macro avg       1.00      1.00      1.00        32
weighted avg       1.00      1.00      1.00        32

